Chapter 1, paragraph 1

Measures of central tendency

1.1 - the arithmetic mean


When we talk about statistics, the most natural starting point for the journey is the arithmetic mean. As every simple concept, it is very powerfull and it is something that - trascending the statistics - we, as social animals, have to deal with daily. Don't we always have in mind an "average something" against which all the rest is compared? But aside from this digression, from a statistical perspective the mean is the most natural and straight forward way to describe a complex phenomenom. Imagine you have to answer the question: in the foru largest European countries, are women or men the more frequent investor in adult education? Let's see how to answer this question using data from the OECD.

In [12]:
import sys
import os
import getpass

if getpass.getuser() == 'simon':
    functionFolder = 'C:/Users/simon/Desktop/github_projects/hands_on_statistics/zz_help_functions/'
else:
    functionFolder = '/Users/tugbasabanoglu/Desktop/hands_on_statistics/'

sys.path.insert(1, functionFolder)
from database_connection_functions import get_from_oecd


# Let's load the dataset and narrow it down to the values we are interested in

adult_education_df = get_from_oecd('EAG_AL_26092022')
adult_education_df = adult_education_df[(adult_education_df['COUNTRY'].isin(['DEU', 'FRA', 'ITA', 'GBR'])) & 
                                        (adult_education_df['AGE'] == 'T') & 
                                        (adult_education_df['MEASURE'] == 'VALUE') & 
                                        (adult_education_df['INDICATOR'] == 'AES_AL_PART_FNFAET') &
                                        (adult_education_df['ISC11A'] == 'T')].reset_index(drop = True)

adult_education_mean_df = adult_education_df.groupby(['Gender']).agg({'Value': 'mean'}).reset_index(drop = False)
adult_education_mean_df


,Gender,Value
0,Men,48.75
1,Total,49.25
2,Women,49.75


Women do seem to be, on average in the 4 largest European countries, make use of adult education more than men. 49.75% of women against 48.75% of men. 
Is this a piece of knowledge we can really make use for? Usually, the best way to make use of the arithmetic mean is to show it in contrast to the single datapoints that generated them. And here is where data visualization becomes a powerfull tool and ally. Let's continue the example from above to see how.